In [ ]:
""" Return list of rotations of input string t """
def Rotations(t):
    tt = t * 2
    return [tt[i : i + len(t)] for i in range(0, len(t))]

In [ ]:
""" Return lexicographically sorted list of t's rotations """
def BWM(t):
    return sorted(Rotations(t))

In [ ]:
""" Given T, returns BWT(T) (last column) by creating BWM """
def BWTViaBWM(t):
    return ''.join(map(lambda x: x[-1], BWM(t)))

In [ ]:
""" Given T return suffix array SA(T) """
def SuffixArray(s):
    satups = sorted([(s[i:], i) for i in range(len(s))])
    # Extract and return just the offsets
    return list(map(lambda x: x[1], satups))

In [ ]:
""" Given T, returns BWT(T) (last column) by way of the suffix array """
def BWTViaSA(t):
    bw = []
    for si in SuffixArray(t):
        if si == 0:
            bw.append('$')
        else:
            bw.append(t[si - 1])
    return ''.join(bw) # returns string version of list bw

In [ ]:
""" Given BWT string bw, return parallel list of B-ranks. Also
    return tots: map from character to # times it appears. """
def RankBWT(bw):
    tots = dict()
    ranks = []
    for c in bw:
        if c not in tots:
            tots[c] = 0
        ranks.append(tots[c])
        tots[c] += 1
    return ranks, tots

In [ ]:
""" Return map from character to the range of rows prefixed by 
    the character. """
def FirstColumn(tots):
    first = {}
    totc = 0
    for c, count in sorted(tots.items()):
        first[c] = (totc, totc + count)
        totc += count
    return first

In [ ]:
""" Make T from BWT(T) """
def ReverseBWT(bw):
    ranks, tots = RankBWT(bw)
    first = FirstColumn(tots)
    rowi = 0   # first row
    t = '$'    # rightmost character
    while bw[rowi] != '$':
        c = bw[rowi]
        t = c + t    # prepend to answer
        # jump to row that starts with c of same rank
        rowi = first[c][0] + ranks[rowi]
    return t

In [ ]:
"""Return the min and max ranks of a specified caracter from the BWT"""
def SetRank(ranks, lColumn, char, lowerIndex, higherIndex):
    indexesOfChar = []
    
    for i in range(lowerIndex, higherIndex):#lower index is inclusive, higher is exclusive
        if lColumn[i] == char:
            indexesOfChar.append(i)
    
    if indexesOfChar:
        lowerIndex = min(indexesOfChar)
        higherIndex = max(indexesOfChar)
    else:
        return (-1, -1)
    
    return (ranks[lowerIndex], ranks[higherIndex])

In [ ]:
"""Return the range of indexes for a specified caracter with given ranks from the First Column of BWM"""
def SetIndex(fColumn, char, lowerRank, higherRank):
    lowerIndex = fColumn[char][0] + lowerRank
    higherIndex = fColumn[char][0] + higherRank + 1 #+1 is to make higherIndex exclusive
    return (lowerIndex, higherIndex)

In [ ]:
""" Return positions where the pattern matches the sequence, otherwise throw exception """
def SearchViaBWM(sequnce, pattern):
    lColumn = BWTViaBWM(sequnce)
    ranks, tots = RankBWT(lColumn)
    fColumn = FirstColumn(tots)
    positions = SuffixArray(sequnce)
    
    lowerIndex = 0
    higherIndex = 0
    lowerRank = 0
    higherRank = 0
    firstIteration = True
    
    for char in reversed(pattern):
        if firstIteration:
            firstIteration = False
            if fColumn.get(char) is None:
                return [-1]
            else:
                (lowerIndex, higherIndex) = fColumn[char]
                continue
        (lowerRank, higherRank) = SetRank(ranks, lColumn, char, lowerIndex, higherIndex)
        if lowerRank == -1 or higherRank == -1:
            return [-1]
        else:
            (lowerIndex, higherIndex) = SetIndex(fColumn, char, lowerRank, higherRank)
    
    return [positions[i] for i in range(lowerIndex, higherIndex)]

In [ ]:
""" Return positions where the pattern matches the sequence, otherwise throw exception """
def SearchViaOldSA(sequnce, pattern):
    lColumn = BWTViaSA(sequnce)
    ranks, tots = RankBWT(lColumn)
    fColumn = FirstColumn(tots)
    positions = SuffixArray(sequnce)
    
    lowerIndex = 0
    higherIndex = 0
    lowerRank = 0
    higherRank = 0
    firstIteration = True
    
    for char in reversed(pattern):
        if firstIteration:
            firstIteration = False
            if fColumn.get(char) is None:
                return [-1]
            else:
                (lowerIndex, higherIndex) = fColumn[char]
                continue
        (lowerRank, higherRank) = SetRank(ranks, lColumn, char, lowerIndex, higherIndex)
        if lowerRank == -1 or higherRank == -1:
            return [-1]
        else:
            (lowerIndex, higherIndex) = SetIndex(fColumn, char, lowerRank, higherRank)
    
    return [positions[i] for i in range(lowerIndex, higherIndex)]